It was only possible to achieve an accuracy of 0.33; further manipulation with hyperparameters does not lead to anything in particular. Maybe resnet50 model will be much better, it should be tested

# Install

In [64]:
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.14.0
  latest version: 23.9.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

Retrieving notices: ...working... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-auzaxbzb
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-auzaxbzb
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done


## Imports

In [65]:
import numpy as np
import pandas as pd

import os
import shutil
import torch
import torchvision
from torchvision import transforms

from tqdm.notebook import tqdm

from torch import nn
from torch.nn import functional as F

from torch.utils.data import DataLoader
from torch.utils.data import Subset

from IPython.display import clear_output

import matplotlib.pyplot as plt


import pandas as pd
import numpy as np
import torch, transformers, time, datetime, random

from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler

import os
import shutil
import torch
import torchvision

from tqdm.notebook import tqdm

from torch import nn
from torch.nn import functional as F

from torch.utils.data import DataLoader
from torch.utils.data import Subset

from IPython.display import clear_output

import matplotlib.pyplot as plt
print(0)

0


In [66]:
# GPU useage

if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
    

def gpuon(x:torch.tensor) -> bool:
    # 텐서가 GPU에 있는지 확인
    is_cuda = x.is_cuda
    if is_cuda:
        # 텐서가 GPU에 있으면 다음 출력
        print("GPU ON.")
    else:
        # 텐서가 CPU에 있으면 다음 출력
        print("CPU ON.")

No GPU available, using the CPU instead.


## Load data

In [67]:
data = pd.read_csv('/kaggle/input/youtube-thumbnail-dataset/thumbnails/metadata.csv')

cat_map = {}
cat_list = data["Category"].unique().tolist()
for i in range(len(cat_list)):
    cat_map[cat_list[i]] = i


# cat_map : index of category 
# cat_list : list of category 


data["cat_code"] = data["Category"].apply(lambda x: cat_map[x])

data.cat_code # 카테고리를 인덱스로 추가

data

,Id,Channel,Category,Title,cat_code
0,OkmNXy7er84,3Blue1Brown,Science,The hardest problem on the hardest test,0
1,r6sGWTCMz2k,3Blue1Brown,Science,But what is a Fourier series? From heat flow t...,0
2,bBC-nXj3Ng4,3Blue1Brown,Science,But how does bitcoin actually work?,0
3,aircAruvnKk,3Blue1Brown,Science,"But what is a neural network? | Chapter 1, Dee...",0
4,HEfHFsfGXjs,3Blue1Brown,Science,The most unexpected answer to a counting puzzle,0
...,...,...,...,...,...
2510,In0nB0ABaUk,Web Dev Simplified,Tech,JavaScript Form Validation,4
2511,y17RuWkWdn8,Web Dev Simplified,Tech,Learn DOM Manipulation In 18 Minutes,4
2512,O6P86uwfdR0,Web Dev Simplified,Tech,Learn useState In 15 Minutes - React Hooks Exp...,4
2513,DUg2SWWK18I,Web Dev Simplified,Tech,MVC Explained in 4 Minutes,4


## Prepare data

In [68]:
!cp -r /kaggle/input/youtube-thumbnail-dataset/thumbnails/images/ ./

# prepare images

In [8]:
# basedir = "/kaggle/working/images/"

# for fn in os.listdir(basedir):
#     if fn not in ch2cat.keys(): continue
#     try:
#         os.rename(os.path.join(basedir, fn), os.path.join(basedir, ch2cat[fn]))
#     except OSError:
#         shutil.move(os.path.join(basedir, fn), os.path.join(basedir, ch2cat[fn]))

In [34]:
import os

def print_directory_structure(path):
  for root, directories, files in os.walk(path):
    for directory in directories:
      print(f"{root}/{directory}")
    for file in files:
      print(f"{root}/{file}")

# print_directory_structure(".")

# Modules

In [70]:

import clip,torch
import requests
from PIL import Image
import numpy as np
import torch
from io import BytesIO

label = ["Science",
"News",
"Food",
"Blog",
"Tech",
"Informative",
"Comedy,Entertainment",
"Entertainment",
"Automobile",
"Tech,Informative",
"Automobile,Comedy",
"VideoGames",
"Food,Entertainment",
"Blog,Comedy",
"Comedy,Informative",
"Tech,Comedy",
"Comedy",
"Blog,Science",
"Blog,Entertainment",
"Entertainment,Comedy",
"Tech,News",
"Entertainment,Blog"]

# TODO: cosine sims

def cosine_sim(x_emb,label_emb : torch.tensor) -> torch.tensor: # vector -> scalar 
    x_emb = x_emb.view(512)
    
    scores = torch.dot(x_emb,label_emb[0].view(512)).view(1)
    
    for i in range(1,22):
        label_emb_i = label_emb[i].view(512)
        score = torch.dot(x_emb,label_emb_i)
        score = score.view(1)
        scores = torch.cat([scores,score],dim=0)

    return scores

# TODO: softmax
def softmax(img_sim):
    return torch.exp(img_sim) / torch.sum(torch.exp(img_sim))

# TODO: clip probs

def label_embedding(label:list) -> torch.tensor:
    """Gets the embedding values for the image."""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, preprocess = clip.load("ViT-B/32", device=device)
    label_token = clip.tokenize(label).to(device)
    # torch.Size([22, 77])
    
    with torch.no_grad():
        # encoding
        label_features = model.encode_text(label_token)
        # torch.Size([# of text, 512] 22
    return label_features

label_features = label_embedding(label)

def clip_probs(jpg,text,jpg_type=0):
    global label_features
    
    """Gets the embedding values for the image."""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, preprocess = clip.load("ViT-B/32", device=device)
    
    if jpg_type==0:
        
        response = requests.get(jpg)
        image_bytes = response.content
        
        #Url 이미지 전처리.
        preprocess_image = preprocess(Image.open(BytesIO(image_bytes))).unsqueeze(0).to(device)
        # torch.Size([1, 3, 224, 224])
        
    else:
        # 디텍토리 jpg 이미지 처리
        # ex .../CLIP.jpg
        preprocess_image = preprocess(Image.open(jpg)).unsqueeze(0).to(device) 


    # 텍스트 tokenize
    text_token = clip.tokenize(text).to(device)
    # torch.Size([7, 77])


    # output.dtype : torch.float32

    with torch.no_grad():
        # encoding
        image_features = model.encode_image(preprocess_image)
        # torch.Size([# of img, 512]  1
        text_features = model.encode_text(text_token)
        # torch.Size([# of text, 512] 1
        
        
        # logits_per_image, logits_per_text = model(preprocess_image,text_token)
        # probs = logits_per_image.softmax(dim=-1).cpu().numpy()
        # print('labels probability',probs)
        
        
        ## similarity of image
        img_sim = cosine_sim(image_features,label_features)
        text_sim = cosine_sim(text_features,label_features)    

        # return text_features,image_features,probs


    #img_probs = label[torch.argmax(softmax(img_sim))]
    #text_probs = label[torch.argmax(softmax(text_sim))]
    # print(f'image similarity : {img_probs}')
    # print(f'text similarity : {text_probs}')
    
    #return img_probs,text_probs

    
    ##### TOP K #######
    
    topk = 3
    #torch.topk output types
    #torch.return_types.topk(
    #values=tensor([1.7843, 1.6211, 1.4115]),
    #indices=tensor([7, 8, 0])),

    img_topk_probs = list(torch.topk(softmax(img_sim),k=topk)[0].numpy())  # list 로 바꾸기
    text_topk_probs = list(torch.topk(softmax(text_sim),k=topk)[0].numpy()) #
    
    img_topk_index = torch.topk(softmax(img_sim),k=topk)[1]
    text_topk_index = torch.topk(softmax(text_sim),k=topk)[1]
    
    img_topk_cat = [label[img_topk_index[i]] for i in range(topk)]
    text_topk_cat = [label[text_topk_index[i]] for i in range(topk)]
    
    img_result = {'probs':img_topk_probs,'cat':img_topk_cat}
    text_result = {'probs':text_topk_probs,'cat':text_topk_cat}
    return img_result,text_result



In [ ]:

import clip,torch
import requests
from PIL import Image
import numpy as np
import torch
from io import BytesIO


X_title = data.Title.apply(lambda x: ''.join([i for i in x if not i.isdigit()])).tolist() # no numbers
y = data.cat_code.tolist()

dir = "/kaggle/working/images/"

counts = 0
img_counts = 0
text_counts = 0
total =  2515

from tqdm import tqdm

result_dict = {}
counts = 0

channels = {}
solution = {}

for i in list(data.Channel.unique()): # channel's name
    channels[i] = {'total':0,'counts':0,'img_counts':0,'text_counts':0}

for i in list(data.Channel.unique()): # channel's name
    solution[i] = {'label':[],'image':[],'text':[]}



for i in tqdm(range(80,90)):
    # print(f'{i}번째 입니다!')
    ch = data['Channel'][i]
    id = data['Id'][i]
    cat = data['Category'][i]
    cat_index = data['cat_code'][i]
    
    x_image_dir = f'{dir}{ch}/{id}.jpg'
    # print(image_dir)
    # x_image = Image.open(x_image_dir) # jpg
    x_title = X_title[i] # text
    x_label = y[i] # label 
    
    try:
        img_result, text_result= clip_probs(x_image_dir,x_title,jpg_type=1)
        #img_result = {'probs':img_topk_probs,'cat':img_topk_cat}
        
        #topk까지 맞는지 확인
        img_topk = False
        text_topk = False
        
        print('image',img_result)
        print('text',text_result)
        print('label',cat)
        print()
        
        # images
        if cat in img_result['cat']:
            img_topk = True
            channels[ch]['img_counts']+=1 # 딕셔너리 채널에 추가
            img_counts +=1 # 전체 이미지가 맞는 개수

        # text
        if cat in text_result['cat']:
            text_topk = True
            channels[ch]['text_counts']+=1 # dictionary 채널에 추가 
            text_counts +=1 # 전체 텍스트 정답률
            
        # total
        if img_topk or text_topk:
            counts+=1        # 전체 맞는개수
            channels[ch]['counts']+=1 # 채널에서 맞는개수
            
            
    except:
        print('no data error!')

 10%|█         | 1/10 [00:05<00:47,  5.31s/it]

image {'probs': [0.21244374, 0.2002664, 0.19593194], 'cat': ['Food,Entertainment', 'Food', 'Blog']}
text {'probs': [nan, nan, nan], 'cat': ['Comedy,Informative', 'Comedy', 'Tech,Comedy']}
label Food



 20%|██        | 2/10 [00:10<00:39,  4.96s/it]

image {'probs': [0.53248346, 0.1137125, 0.085588485], 'cat': ['Blog,Comedy', 'Blog,Entertainment', 'Blog,Science']}
text {'probs': [0.3776674, 0.3034643, 0.15068372], 'cat': ['Blog,Comedy', 'Comedy', 'Blog,Science']}
label Food



 30%|███       | 3/10 [00:15<00:35,  5.02s/it]

image {'probs': [0.31495374, 0.20249444, 0.1684399], 'cat': ['Blog,Comedy', 'Food,Entertainment', 'Blog,Entertainment']}
text {'probs': [0.39078715, 0.32171038, 0.1705218], 'cat': ['Comedy', 'Blog,Comedy', 'Entertainment,Comedy']}
label Food



 40%|████      | 4/10 [00:19<00:29,  4.95s/it]

image {'probs': [0.29258332, 0.26073825, 0.13934502], 'cat': ['Blog,Science', 'Blog,Comedy', 'Food,Entertainment']}
text {'probs': [0.6377209, 0.17169133, 0.09273365], 'cat': ['Comedy', 'Entertainment,Comedy', 'Blog,Comedy']}
label Food



In [64]:
dir = "/kaggle/working/images/"

result_dict = {}
channels = {}

for i in list(data.Channel.unique()): # channel's name
    channels[i] = {'total':0,'counts':0,'img_counts':0,'text_counts':0}

for i in tqdm(range(10)): # 2515
    # print(f'{i}번째 입니다!')
    ch = data['Channel'][i]
    id = data['Id'][i]
    cat = data['Category'][i]
    print(ch)
    print(id)
    print(cat)
    
    x_title = data['Title'][i] # 숫자 포함.
    # x_title = X_title[i] # text 숫자 불포함.
    
    x_label = data['cat_code'][i]
    # x_label = y[i] # label 
    
    print(x_title)
    print(x_label)
    # 해당 채널의 수 증가.
    channels[ch]['total']+=1
    
    
    x_image_dir = f'{dir}{ch}/{id}.jpg'

    img_prob, text_prob = clip_probs(x_image_dir,x_title,jpg_type=1)

print(img_prob)
print(text_prob)
print(device)

  0%|          | 0/10 [00:00<?, ?it/s]

3Blue1Brown
OkmNXy7er84
Science
The hardest problem on the hardest test
0


 10%|█         | 1/10 [00:04<00:44,  4.96s/it]

GPU ON.
GPU ON.
3Blue1Brown
r6sGWTCMz2k
Science
But what is a Fourier series? From heat flow to drawing with circles | DE4
0


 20%|██        | 2/10 [00:09<00:37,  4.65s/it]

...

GPU ON.
GPU ON.
3Blue1Brown
GNcFjFmqEc8
Science
But why is a sphere's surface area four times its shadow?
0


 90%|█████████ | 9/10 [00:41<00:04,  4.71s/it]

GPU ON.
GPU ON.
3Blue1Brown
fNk_zzaMoSs
Science
Vectors | Chapter 1, Essence of linear algebra
0


100%|██████████| 10/10 [00:46<00:00,  4.63s/it]

GPU ON.
GPU ON.
Science
Science
cuda


## 지피유 사용테스트


## clip test

In [ ]:
import clip,torch
import requests
from PIL import Image
import numpy as np
import torch
from io import BytesIO
from tqdm import tqdm

# dir = '/Users/juniverse/Desktop/pointcloud/VectorUniverse/Data/thumbnails/images/'
dir = "/kaggle/working/images/"



result_dict = {}

channels = {}
solution = {}

for i in list(data.Channel.unique()): # channel's name
    channels[i] = {'total':0,'counts':0,'img_counts':0,'text_counts':0}

for i in list(data.Channel.unique()): # channel's name
    solution[i] = {'label':[],'image':[],'text':[]}

for i in tqdm(range(2515)): # 2515
    # print(f'{i}번째 입니다!')
    ch = data['Channel'][i]
    id = data['Id'][i]
    cat = data['Category'][i]
    
    x_title = data['Title'][i] # 숫자 포함.
    # x_title = X_title[i] # text 숫자 불포함.
    
    x_label = data['cat_code'][i]
    # x_label = y[i] # label 
    
    # 해당 채널의 수 증가.
    channels[ch]['total']+=1
    
    
    x_image_dir = f'{dir}{ch}/{id}.jpg'
    try:
        img_prob, text_prob = clip_probs(x_image_dir,x_title,jpg_type=1)
        
        print('image :',img_prob)
        print('text :',text_prob)
        print('label :',cat)
        try:
            solution[ch]['label'].append(cat)
            solution[ch]['image'].append(img_prob)
            solution[ch]['text'].append(text_prob)
        except:
            print('solution error!')
    except:
        print('no data error!')


  0%|          | 1/2515 [00:04<3:05:15,  4.42s/it]

GPU ON.
GPU ON.
image : Science
text : Science
label : Science


  0%|          | 2/2515 [00:08<3:05:53,  4.44s/it]

GPU ON.
GPU ON.
image : Science
text : Science
label : Science


  0%|          | 3/2515 [00:13<3:02:32,  4.36s/it]

...

no data error!


 11%|█         | 280/2515 [20:29<1:43:39,  2.78s/it]

no data error!


 11%|█         | 281/2515 [20:34<2:05:45,  3.38s/it]

GPU ON.
GPU ON.
image : Science
text : Science
label : Entertainment


{'3Blue1Brown': {'label': ['Science',
   'Science',
   'Science',
   'Science',
   'Science'],
  'image': ['Science', 'Science', 'Science', 'Science', 'Science'],
  'text': ['Science', 'Science', 'Science', 'Science', 'Science']},
 'A&E': {'label': [], 'image': [], 'text': []},
 'About To Eat': {'label': [], 'image': [], 'text': []},
 'Abroad in Japan': {'label': [], 'image': [], 'text': []},
 'Austin Evans': {'label': [], 'image': [], 'text': []},
 'Barely Sociable': {'label': [], 'image': [], 'text': []},
 'BBC News': {'label': [], 'image': [], 'text': []},
 'Bon Appétit': {'label': [], 'image': [], 'text': []},
 'Brooklyn Nine-Nine': {'label': [], 'image': [], 'text': []},
 'BuzzFeedVideo': {'label': [], 'image': [], 'text': []},
 'Captain Disillusion': {'label': [], 'image': [], 'text': []},
 'Car Throttle': {'label': [], 'image': [], 'text': []},
 'carwow': {'label': [], 'image': [], 'text': []},
 'CDawgVA': {'label': [], 'image': [], 'text': []},
 'ChrisFix': {'label': [], 'image

In [ ]:
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

# based on https://gist.github.com/srikarplus/8bdb5bedf0ca25e894e39ea78fce2f39
def get_train_valid_loader(data_dir,
                           batch_size,
                           augment,
                           random_seed,
                           valid_size=0.1,
                           shuffle=True,
                           show_sample=False,
                           num_workers=4,
                           pin_memory=False):
    """
    
    Params
    ------
    - data_dir: path directory to the dataset.
    - batch_size: how many samples per batch to load.
    - augment: whether to apply the data augmentation scheme
      mentioned in the paper. Only applied on the train split.
    - random_seed: fix seed for reproducibility.
    - valid_size: percentage split of the training set used for
      the validation set. Should be a float in the range [0, 1].
    - shuffle: whether to shuffle the train/validation indices.
    - show_sample: plot 9x9 sample grid of the dataset.
    - num_workers: number of subprocesses to use when loading the dataset.
    - pin_memory: whether to copy tensors into CUDA pinned memory. Set it to
      True if using GPU.
    Returns
    -------
    - train_loader: training set iterator.
    - valid_loader: validation set iterator.
    """
    error_msg = "[!] valid_size should be in the range [0, 1]."
    assert ((valid_size >= 0) and (valid_size <= 1)), error_msg

    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    )

    # define transforms
    valid_transform = transforms.Compose([transforms.Resize(512),
                                      transforms.ToTensor(),
                                      normalize])

    train_transform = transforms.Compose([transforms.Resize(512),
                                      transforms.ToTensor(),
                                      normalize])

    # load the dataset
    train_dataset = datasets.ImageFolder(
        root=data_dir, transform=train_transform,
    )

    valid_dataset = datasets.ImageFolder(
        root=data_dir, transform=valid_transform,
    )

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    )
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    )


    return (train_loader, valid_loader, train_dataset.class_to_idx)

In [ ]:
train_loader, test_loader, class_idx = get_train_valid_loader('/kaggle/working/images/', batch_size, False, 42, 0.1)

In [ ]:
class_idx

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(10, 10))
for image, number in train_loader:
    for i in range(2):
        for j in range(3):
            ax[i, j].imshow(image[i * 3 + j].permute(1, 2, 0))
            ax[i, j].set_title(f'It\'s {number[i * 3 + j]}')
            ax[i, j].axis('off')
    break
fig.show()

## Prepare model

In [ ]:
class Model(nn.Module):
    def __init__(self, out_features):
        super().__init__()
        self.blocks = nn.Sequential(
            nn.Conv2d(3, 16, 5, 3, 2),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Conv2d(16, 32, 5, 3, 1),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, 3, 1),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, 3, 1),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(512*3*7, 256*3*7),
            nn.ReLU(),
            nn.Linear(256*3*7, 128*3*7),
            nn.ReLU(),
            nn.Linear(128*3*7, out_features)
        )
    
    def forward(self, x):
        return self.blocks(x)

In [ ]:
#!g1.1
model = Model(len(categories))
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
n_epochs = 100

In [ ]:
#!g1.1
model.to(device)

## Train

In [ ]:
from IPython.display import clear_output

def plot_loss_accuracy(train_losses, train_accs, test_accs, n):
    clear_output()
    fig, ax = plt.subplots(1, 2, figsize=(26,10))
    ax[0].set_title(f'Train Loss, epoch {n}')
    ax[0].plot(train_losses)
    ax[1].set_title('Accuracy')
    ax[1].plot(train_accs, label='Train')
    ax[1].plot(test_accs, label='Test')
    ax[1].legend()
    plt.grid()
    plt.show()

In [ ]:
#!g1.1
def train_epoch(model, train_loader, criterion, optimizer, device='cpu'):
    model.train()
    train_losses = []
    train_accuracy = []

    for batch_X, batch_Y in tqdm(train_loader):
        batch_X = batch_X.to(device)
        batch_Y = batch_Y.to(device)

        optimizer.zero_grad()

        softmax = nn.Softmax()
        probs = softmax(model(batch_X))
        classes = torch.argmax(probs, dim=-1)
        loss = criterion(probs, batch_Y)

        loss.backward()

        train_losses.append(loss.item())
        train_accuracy.append((classes == batch_Y).float().mean().item())
        optimizer.step()
    
    return np.mean(train_losses), np.mean(train_accuracy)


def predict(model, test_loader, criterion, device='cpu'):
    model.eval()
    test_accuracy = []
    with torch.no_grad():
        for batch_X, batch_Y in tqdm(test_loader):
            batch_X = batch_X.to(device)
            batch_Y = batch_Y.to(device)

            softmax = nn.Softmax()
            probs = softmax(model(batch_X))
            classes = torch.argmax(probs, dim=-1)
            
            test_accuracy.append((classes == batch_Y).float().mean().item())
    return np.mean(test_accuracy)


def train(model, train_loader, test_loader, criterion, optimizer, n_epochs, device='cpu'):
    train_losses = []
    train_accs = []
    test_accs = []

    for epoch in range(n_epochs):
        loss, accs = train_epoch(model, train_loader, criterion, optimizer, device)
        train_losses.append(loss)
        train_accs.append(accs)
        test_accs.append(predict(model, test_loader, criterion, device))
        plot_loss_accuracy(train_losses, train_accs, test_accs, epoch)


In [ ]:
#!g1.1
import warnings
warnings.filterwarnings("ignore")
train(model, train_loader, test_loader, criterion, optimizer, n_epochs, device)

In [ ]:
#!g1.1
predict(model, test_loader, criterion, device)